In [1]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import HashingVectorizer  
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import nltk.data
from sklearn.metrics.pairwise import cosine_similarity
import time
import pandas as pd


data = pd.read_csv("../example/Posts.csv")
filteredData = data.dropna()
filteredData = filteredData.head(300000)

In [2]:
warnings.filterwarnings(action = 'ignore', category=DeprecationWarning)
postLinks = pd.read_csv("../example/PostLinks.csv")
postLinksDupl = postLinks[postLinks['LinkTypeId'] == 3]
len(postLinksDupl)

1096708

In [3]:
filteredData.head()

,Id,Body,Title,PostTypeId,Tags,CreationDate
0,337,<p>I am about to build a piece of a project th...,XML Processing in Python,1,<python><xml>,2008-08-02T03:35:55.697
1,469,<p>I am using the Photoshop's javascript API t...,How can I find the full path to a font from it...,1,<python><macos><fonts><photoshop>,2008-08-02T15:11:16.430
2,502,<p>I have a cross-platform (Python) applicatio...,Get a preview JPEG of a PDF on Windows?,1,<python><windows><image><pdf>,2008-08-02T17:01:58.500
3,535,<p>I am starting to work on a hobby project wi...,Continuous Integration System for a Python Cod...,1,<python><continuous-integration><extreme-progr...,2008-08-02T18:43:54.787
4,594,<p>There are several ways to iterate over a re...,cx_Oracle: How do I iterate over a result set?,1,<python><sql><database><oracle><cx-oracle>,2008-08-03T01:15:08.507


In [4]:
unansweredQues = pd.read_csv("../example/UnansweredPosts.csv")
print(len(unansweredQues))
unansweredQues['AcceptedAnswerId'].replace(-999, np.nan, inplace=True)
unansweredQues.dropna(subset=['AcceptedAnswerId'], inplace=True)
filUnAns = unansweredQues[unansweredQues['Tags'].str.contains("python")]

1000000


In [5]:
allIds = list(unansweredQues.Id)
print(len(allIds))

unIds = []
for i in filteredData.index:
    if (filteredData['Id'][i] in allIds) and (len(unIds) <= 10):
        unIds.append(filteredData['Id'][i])
    else:
        break

print(unIds)

180080
[337, 469, 502, 535, 594, 683, 742, 766, 773, 972, 1171]


In [6]:
stopwords = stopwords.words('english')
st = WordNetLemmatizer()
def preprocessing(document):
    document_tokenized = [word.lower() for word in word_tokenize(document)]
    document_stopwords = [word for word in document_tokenized if word not in stopwords]
    english_punctuations = [":",".","\n","/","@", "\\","*","=","^",";","_","|",
                               '"',"' "," '","-",
                               "(",")",",",">","<",
                               "!","?","[","]","+",
                               "&","%","$","#","~","{","}"]
    document_filtered = [word for word in document_stopwords if not word in english_punctuations]
    document_stem = [st.lemmatize(word) for word in document_filtered]
    return document_stem

In [7]:
combinedText = []
combinedTextPreprocsd = []
count = 0
for i in filteredData.index:
    count = count+1
    text = str(filteredData['Title'][i]) + " " + str(filteredData['Body'][i])
    combinedText.append(text)
    combinedTextPreprocsd.append(preprocessing(text))
    if(count % 50000 == 0):
        print("Status of preprocessing text: " + str(count))
        
filteredData['tokenized_title'] = combinedTextPreprocsd
filteredData['combined_text'] = combinedText

filteredData.head()

Status of preprocessing text: 50000
Status of preprocessing text: 100000
Status of preprocessing text: 150000
Status of preprocessing text: 200000
Status of preprocessing text: 250000
Status of preprocessing text: 300000


,Id,Body,Title,PostTypeId,Tags,CreationDate,tokenized_title,combined_text
0,337,<p>I am about to build a piece of a project th...,XML Processing in Python,1,<python><xml>,2008-08-02T03:35:55.697,"[xml, processing, python, p, build, piece, pro...",XML Processing in Python <p>I am about to buil...
1,469,<p>I am using the Photoshop's javascript API t...,How can I find the full path to a font from it...,1,<python><macos><fonts><photoshop>,2008-08-02T15:11:16.430,"[find, full, path, font, display, name, mac, p...",How can I find the full path to a font from it...
2,502,<p>I have a cross-platform (Python) applicatio...,Get a preview JPEG of a PDF on Windows?,1,<python><windows><image><pdf>,2008-08-02T17:01:58.500,"[get, preview, jpeg, pdf, window, p, cross-pla...",Get a preview JPEG of a PDF on Windows? <p>I h...
3,535,<p>I am starting to work on a hobby project wi...,Continuous Integration System for a Python Cod...,1,<python><continuous-integration><extreme-progr...,2008-08-02T18:43:54.787,"[continuous, integration, system, python, code...",Continuous Integration System for a Python Cod...
4,594,<p>There are several ways to iterate over a re...,cx_Oracle: How do I iterate over a result set?,1,<python><sql><database><oracle><cx-oracle>,2008-08-03T01:15:08.507,"[cx_oracle, iterate, result, set, p, several, ...",cx_Oracle: How do I iterate over a result set?...


In [8]:
dictTitletoId = {}
for i in filteredData.index:
    dictTitletoId[filteredData['Title'][i]] = filteredData['Id'][i]
        
dictIdToTitle = {}
for i in filteredData.index:
    dictIdToTitle[filteredData['Id'][i]] = filteredData['Title'][i]

In [11]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

list_of_token_text = list(sent_to_words(filteredData.tokenized_title))

#tag each document 
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(list_of_token_text)]
modelDoc2Vec = Doc2Vec(documents, vector_size=300, window=1, 
                min_count=3, workers=4, dm=1, alpha = 0.025,
                sample= 0.01, dm_concat = 1, dbow_words = 1)
modelDoc2Vec.save("model_doc2vec_python.model")

In [18]:
def itemDoc2Vec(id):
    try:
        title = filteredData.loc[filteredData['Id'] == id]['Title'].tolist()[0]
    except:
        title = unansweredQues.loc[unansweredQues['Id'] == id]['Title'].tolist()[0]
    return title

def recommendDoc2Vec(id):
    print("Recommending posts similar to: " + itemDoc2Vec(id)) 
    print("----------------------------------------------------------")
    title_input = ''
    body_input = ''
    for i in range(0, len(filteredData)):
        if filteredData['Id'][i] == id:
            title_input = filteredData['Title'][i]
            body_input = filteredData['Body'][i]
            break
    if title_input == '' or body_input == '':
        title_input = unansweredQues.loc[unansweredQues['Id'] == id]['Title'].tolist()[0]
        body_input = unansweredQues.loc[unansweredQues['Id'] == id]['Body'].tolist()[0]
    test_input = title_input + " " + body_input
    test_input = preprocessing(test_input)
    test_vector = modelDoc2Vec.infer_vector(test_input)
    similar_cases = modelDoc2Vec.docvecs.most_similar(positive = [test_vector], topn=11)
    top_10_similar_cases = similar_cases [1:11]
    print("You may like to read: \n")
    for i in range(0, len(top_10_similar_cases)):
        print("PostId: "+ str(filteredData.loc[top_10_similar_cases[i][0]]['Id']) +
          ". "+ filteredData.loc[top_10_similar_cases[i][0]]['Title'] + " (score:" + str(round(top_10_similar_cases[i][1], 3)) + ")")
    return top_10_similar_cases[0][1]

In [32]:
simScoreListDoc2Vec = []
for uniqid in unIds:
    simScore = recommendDoc2Vec(uniqid) #Passing the postId of an unanswered post
    simScoreListDoc2Vec.append("Highest score for Post Id " + str(uniqid) + " is = " + str(simScore))
print(*simScoreListDoc2Vec, sep = "\n")

Recommending posts similar to: XML Processing in Python
----------------------------------------------------------
You may like to read: 

PostId: 3599569. Automatic editor of XML (based on XSD scheme) (score:0.617)
PostId: 9141062. Obtaining the model topology from a Simulink model (score:0.567)
PostId: 12883428. How to parse restructuredtext in python? (score:0.514)
PostId: 2836132. Python modify an xml file (score:0.511)
PostId: 8837950. PyQt - XML to QTreeWidget (score:0.508)
PostId: 804992. How to consume XML from RESTful web services using Django / Python? (score:0.506)
PostId: 1227031. What is a good configuration file library for c thats not xml (preferably has python bindings)? (score:0.495)
PostId: 2442014. Tree libraries in python (score:0.495)
PostId: 4079829. Generating Python code from XML tree (score:0.491)
PostId: 12965760. how do I search for an xml tag with the matching closing tag? (score:0.485)
Recommending posts similar to: How can I find the full path to a font fr

You may like to read: 

PostId: 53225. How do you check whether a python method is bound or not? (score:0.54)
PostId: 18632520. Aceess a methods docstring without instance, Python (score:0.517)
PostId: 20064576. what does the term 'bound' means in class method (score:0.497)
PostId: 15485655. How can use python to arrange objects in a circular pattern? (score:0.482)
PostId: 11741919. Utilizing two Redis instances - Similar to Mongos (score:0.482)
PostId: 10882469. What's the Groovy equivalent to Python's dir()? (score:0.456)
PostId: 7559170. What's the difference between setattr() and object.__setattr__()? (score:0.456)
PostId: 5029352. What is python __get__ method good for? (score:0.456)
PostId: 1047021. Overriding "+=" in Python? (__iadd__() method) (score:0.451)
PostId: 22345558. What methods does an 'object' has in python? (score:0.443)
Recommending posts similar to: What is the most efficient graph data structure in Python?
---------------------------------------------------------

In [11]:
#Testing purpose
modelDoc2Vec = gensim.models.Doc2Vec.load('model_doc2vec_python.model')
modelDoc2Vec.most_similar('csv')

<ipython-input-11-3cfa0be3b49d>:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  modelDoc2Vec.most_similar('csv')


[('tsv', 0.6410951614379883),
 ('xlsx', 0.600398063659668),
 ('xls', 0.5990321636199951),
 ('dbf', 0.5860942602157593),
 ('unicodecsv', 0.5680943727493286),
 ('arff', 0.5679473876953125),
 ('pkl', 0.5340641140937805),
 ('writerows', 0.529883861541748),
 ('readasdataurl', 0.5289368629455566),
 ('kml', 0.518033504486084)]

In [ ]:
VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}
def preprocess_tf_idf(text):
    text = text.lower()
    temp_sent =[]
    words = nltk.word_tokenize(text)
    tags = nltk.pos_tag(words)
    for i, word in enumerate(words):
        if tags[i][1] in VERB_CODES: 
            lemmatized = st.lemmatize(word, 'v')
        else:
            lemmatized = st.lemmatize(word)
        if lemmatized not in stopwords and lemmatized.isalpha():
            temp_sent.append(lemmatized)
          
    finalsent = ' '.join(temp_sent)
    finalsent = finalsent.replace("n't", " not")
    finalsent = finalsent.replace("'m", " am")
    finalsent = finalsent.replace("'s", " is")
    finalsent = finalsent.replace("'re", " are")
    finalsent = finalsent.replace("'ll", " will")
    finalsent = finalsent.replace("'ve", " have")
    finalsent = finalsent.replace("'d", " would")
    return finalsent
  
filteredData["combined_text_processed"]= filteredData['combined_text'].apply(preprocess_tf_idf)

filteredData.head()

In [22]:
from sklearn.metrics.pairwise import linear_kernel
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=50, stop_words='english')
filteredData_new = filteredData.head(10000)

tfidf_matrix = tf.fit_transform(filteredData_new['combined_text_processed'])
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
results = {} # dictionary created to store the result in a dictionary format (ID : (Score,item_id))#

for idx, row in filteredData_new.iterrows(): 
    # the below code 'similar_indice' stores similar ids based on cosine similarity. sorts them in ascending order. [:-10:-1] is then used so that the indices with most similarity are got. 0 means no similarity and 1 means perfect similarity#
    similar_indices = cosine_similarities[idx].argsort()[:-10:-1] 

    #stores 10 most similar books, you can change it as per your needs
    similar_items = [(cosine_similarities[idx][i], filteredData_new['Id'][i]) for i in similar_indices]
    results[row['Id']] = similar_items[1:]

In [24]:
def itemTfIdf(id):
    try:
        title = filteredData_new.loc[filteredData_new['Id'] == id]['Title'].tolist()[0]
    except:
        title = unansweredQues.loc[unansweredQues['Id'] == id]['Title'].tolist()[0]
    return title

def recommendTfIdf(id):
    print("Recommending posts similar to: " + itemTfIdf(id)) 
    print("----------------------------------------------------------")
    recs = results[id][:11]
    print("You may like to read: \n")
    for rec in recs:
        print("Post Id:" + str(dictTitletoId[itemTfIdf(rec[1])]) + ". " + itemTfIdf(rec[1]) + " (score:" + str(round(rec[0], 3)) + ")")
    return recs[0][0]

In [34]:
simScoreListTfIdf = []
for uniqid in unIds:
    simScore = recommendTfIdf(uniqid) #Passing the postId of an unanswered post
    simScoreListTfIdf.append("Highest score for Post Id " + str(uniqid) + " is = " + str(simScore))
print(*simScoreListTfIdf, sep = "\n") 

Recommending posts similar to: XML Processing in Python
----------------------------------------------------------
You may like to read: 

Post Id:307616. XML instance generation from XML schema (xsd) (score:0.56)
Post Id:299588. Validating with an XML schema in Python (score:0.555)
Post Id:1250192. parsing XML file in python with cElementTree: dealing with errors and line number in the file (score:0.553)
Post Id:982414. How can I search and replace in XML with Python? (score:0.548)
Post Id:259782. Does Python 2.5 include a package to natively transform an XML document? (score:0.528)
Post Id:1294654. Getting DOM tree of XML document (score:0.515)
Post Id:321795. Comparing XML in a unit test in Python (score:0.51)
Post Id:127606. Editing XML as a dictionary in python? (score:0.504)
Recommending posts similar to: How can I find the full path to a font from its display name on a Mac?
----------------------------------------------------------
You may like to read: 

Post Id:872054. Python 

In [13]:
corpus = []
for words in filteredData['combined_text_processed'][:100000]:
    corpus.append(words.split())
modelWord2Vec = gensim.models.Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
modelWord2Vec.build_vocab(corpus)
modelWord2Vec.train(corpus, total_examples=modelWord2Vec.corpus_count, epochs = 5)

(0, 0)

In [14]:
import requests

filteredDataNewWord2vec = filteredData.head(100000)

# Generate the average word2vec for the each book description

def vectors(x):
    
    # Creating a list for storing the vectors (description into vectors)
    global word_embeddings
    word_embeddings = []

    # Reading the each book description 
    for i in range(0, len(x['combined_text_processed'])):
        avgword2vec = None
        count = 0
        for word in x['combined_text_processed'][i].split():
            if word in modelWord2Vec.wv.vocab:
                count += 1
                if avgword2vec is None:
                    avgword2vec = modelWord2Vec[word]
                else:
                    avgword2vec = avgword2vec + modelWord2Vec[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
        
            word_embeddings.append(avgword2vec)
    
def itemWord2vec(id):
    return filteredDataNewWord2vec.loc[filteredDataNewWord2vec.Id == id]['Title'].tolist()[0]
            

def recommendationsWor2Vec(postId):
    
    # Calling the function vectors

    vectors(filteredDataNewDoc2vec)
    title = itemWord2vec(postId)
    
    # finding cosine similarity for the vectors

    cosine_similarities_word2vec = cosine_similarity(word_embeddings, word_embeddings)

    # taking the title and book image link and store in new data frame called books
    posts = filteredDataNewDoc2vec[['Title', 'Id']]
    #Reverse mapping of the index
    indices = pd.Series(filteredDataNewWord2vec.Id, filteredDataNewWord2vec['Title']).drop_duplicates()
    idx = indices[title]
    print(idx)
    sim_scores = list(enumerate(cosine_similarities_word2vec[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[2:12]
    posts_indices = [i[0] for i in sim_scores]
    recommend = posts.iloc[posts_indices]
    print("Recommending posts similar to: " + title) 
    print("----------------------------------------------------------")
    z = 2
    for index, row in recommend.iterrows():
        try:
            print("Post ID: " + str(row.Id) + ". "+ str(row.Title) + " (" + str(sim_scores[z][1]) + ")")
            z = z + 1
        except:
            continue
    return sim_scores[2][1]

In [ ]:
recommendationsWor2Vec(337)
simScoreListWord2Vec = []
for uniqid in unIds:
    simScore = recommendationsWor2Vec(uniqid) #Passing the postId of an unanswered post
    simScoreListWord2Vec.append("Highest score for Post Id " + str(uniqid) + " is = " + str(simScore))
print(*simScoreListWord2Vec, sep = "\n"  ) 

In [16]:
modelWord2Vec.most_similar('csv')

[('myservice', 0.2544165253639221),
 ('tsp', 0.22140932083129883),
 ('feast', 0.22117266058921814),
 ('glist', 0.21612223982810974),
 ('bfgs', 0.21049481630325317),
 ('toctree', 0.21027931571006775),
 ('boxlinux', 0.20937584340572357),
 ('frameworkname', 0.20935149490833282),
 ('keylist', 0.2092338502407074),
 ('dq', 0.20847007632255554)]